### 환경 setup

In [ ]:
!apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl

In [ ]:
!python3 -m pip install --upgrade pip

In [ ]:
!python3 -m pip install konlpy

In [4]:
from konlpy.tag import Okt
okt = Okt()

In [5]:
import re
import pandas as pd
import numpy as np

### 클롤링 데이터 load

In [6]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [7]:
pwd

'/content'

In [44]:
df1 = pd.read_csv('gdrive/My Drive/dataset_zerowaste/blog.csv')
df1['SNS'] = 'blog'
print(len(df1['Content']))
df1.head(3)

5583


,URL,Date,Like,Content,Hashtag,SNS
0,https://blog.naver.com/lsy_sweet/222457159636,2021. 8. 15. 9:58,12.0,"안녕하세요, 친환경 살림을 하며 환경 보존을 위해 노력하는 단순이 입니다. 그동안 ...",#고체치약 #제로웨이스트 #고체치약본티 #본티치약 #Vontee,blog
1,https://blog.naver.com/newblack76/222468647865,2021. 8. 15. 9:49,NaN,모두들 안녕하셨어요~ 입추가 지나고 말복이 지나니 이젠 바람이 틀려진듯해요^^ 금방...,#국산천연수세미 #제로웨이스트선물 #국내산천연수세미 #북어표수세미 #잘라쓰는수세미 ...,blog
2,https://blog.naver.com/somcandy117/222460526981,2021. 8. 15. 9:10,2.0,오빠가 유리빨대의 딱딱한 질감이 싫다고 해서 100퍼 자연주의 풀빨대를 사봤다 배송...,#풀빨대 #자연빨대 #제로웨이스트 #생분해빨대 #베트남풀빨대 #친환경빨대 #환경보호...,blog


In [45]:
df2 = pd.read_csv('gdrive/My Drive/dataset_zerowaste/instagram.csv')
df2 = df2.drop('Unnamed: 0', axis=1)
df2 = df2.rename(columns={"id": "URL", "date": "Date", "like": "Like", "text":"Content", "hashtag":"Hashtag"})
df2['SNS'] = 'instagram'
print(len(df2['Content']))
df2.head(3)

4500


,URL,Date,Like,Content,Hashtag,SNS
0,vege_yony,2021-08-15 23:59:36+00:00,49,- 그릭요거트 다들 식사로 먹지만 간식으로 먹는 사람 나야 나- 카페 조감 비건그릭...,"['#비건그릭요거트', '#쑥브라우니', '#비건쑥디저트']",instagram
1,eco_kapegg,2021-08-15 23:45:31+00:00,16,⠀ 더운날🥲 음료필수장착해야되쥬 중부사부소 조과장님의 텀블러 용기내👍👏 ⠀ 텀블러에...,"['#용기내', '#제로웨이스트', '#환경보호', '#텀블러', '#캠페인', '...",instagram
2,eco_kapegg,2021-08-15 23:42:02+00:00,14,⠀ 중부사무소 서대리님의 주말 장보기🤗 미리 장바구니를 챙겨서 봉투사용을 줄였습니다...,"['#제로웨이스트', '#캠페인', '#장바구니', '#일회용품줄이기', '#봉투줄...",instagram


In [46]:
df3 = pd.read_csv('gdrive/My Drive/dataset_zerowaste/cafe.csv')
df3['SNS'] = 'cafe'
print(len(df3['Content']))
df3.head(3)

6301


,URL,Date,Like,Content,Hashtag,SNS
0,https://cafe.naver.com/applestore99/1704?art=a...,2021.08.15. 23:25,0.0,​​​써보고 평이 가장 좋았던 갓성비 샴푸바 이구요. 가격후기별 순위모음​​​ <...,NaN,cafe
1,https://cafe.naver.com/myomahealing/151141?art...,2021.08.15. 23:01,3.0,"글쓰기 전 기본 정보가 있는 '공부합시다' 게시판을 확인하시고, 중복되는 문의글이 ...",NaN,cafe
2,https://cafe.naver.com/donggubat0/72?art=aW50Z...,2021.08.15. 22:39,2.0,"동구밭과 첫 인연을 맺게 해준, 그 첫 인연이후 쭉 저의 최애템은 '올바른 설거지 ...",NaN,cafe


In [47]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5583 entries, 0 to 5582
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   URL      5583 non-null   object 
 1   Date     5382 non-null   object 
 2   Like     5190 non-null   float64
 3   Content  5303 non-null   object 
 4   Hashtag  4913 non-null   object 
 5   SNS      5583 non-null   object 
dtypes: float64(1), object(5)
memory usage: 261.8+ KB


In [48]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   URL      4500 non-null   object
 1   Date     4500 non-null   object
 2   Like     4500 non-null   object
 3   Content  4500 non-null   object
 4   Hashtag  4500 non-null   object
 5   SNS      4500 non-null   object
dtypes: object(6)
memory usage: 211.1+ KB


In [49]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6301 entries, 0 to 6300
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   URL      6299 non-null   object 
 1   Date     6299 non-null   object 
 2   Like     6299 non-null   float64
 3   Content  6240 non-null   object 
 4   Hashtag  995 non-null    object 
 5   SNS      6301 non-null   object 
dtypes: float64(1), object(5)
memory usage: 295.5+ KB


### Content 컬럼이 missing value인 row 처리

In [50]:
df1.isnull().sum()

URL          0
Date       201
Like       393
Content    280
Hashtag    670
SNS          0
dtype: int64

In [51]:
df2.isnull().sum()

URL        0
Date       0
Like       0
Content    0
Hashtag    0
SNS        0
dtype: int64

In [52]:
df3.isnull().sum()

URL           2
Date          2
Like          2
Content      61
Hashtag    5306
SNS           0
dtype: int64

In [53]:
def eliminate_nullrows(dataframe):
  dataframe = dataframe.dropna(subset=['Content'])
  dataframe.reset_index(drop=True, inplace=True)
  return dataframe

In [54]:
df1 = eliminate_nullrows(df1)
df2 = eliminate_nullrows(df2)
df3 = eliminate_nullrows(df3)

In [55]:
df1.isnull().sum()

URL          0
Date        93
Like       335
Content      0
Hashtag    545
SNS          0
dtype: int64

In [56]:
df2.isnull().sum()

URL        0
Date       0
Like       0
Content    0
Hashtag    0
SNS        0
dtype: int64

In [57]:
df3.isnull().sum()

URL           0
Date          0
Like          0
Content       0
Hashtag    5245
SNS           0
dtype: int64

### 광고성글 제외시키기

In [58]:
def eliminate_ads(dataframe):
  dataframe_treated = dataframe.query('~Content.str.contains("원고료|광고|홍보|협찬|협찬받아|무상|지원받아|제공받아|제공받고|당첨|이벤트|당첨 후")', engine='python')
  return dataframe_treated

In [59]:
df1 = eliminate_ads(df1)
df2 = eliminate_ads(df2)
df3 = eliminate_ads(df3)

In [60]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4286 entries, 0 to 5301
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   URL      4286 non-null   object 
 1   Date     4209 non-null   object 
 2   Like     4019 non-null   float64
 3   Content  4286 non-null   object 
 4   Hashtag  3830 non-null   object 
 5   SNS      4286 non-null   object 
dtypes: float64(1), object(5)
memory usage: 234.4+ KB


In [61]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5204 entries, 0 to 6239
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   URL      5204 non-null   object 
 1   Date     5204 non-null   object 
 2   Like     5204 non-null   float64
 3   Content  5204 non-null   object 
 4   Hashtag  741 non-null    object 
 5   SNS      5204 non-null   object 
dtypes: float64(1), object(5)
memory usage: 284.6+ KB


각 플랫폼별 8/15기준 최신 4200게시글 분석함. (null content 및 광고성 글 제외)

In [62]:
df_blog = df1[:4200]

In [63]:
df_insta = df2[:4200]

In [64]:
df_cafe = df3[:4200]

### 기본적인 문자 전처리 (띄어쓰기 적용 및 불필요한 기호 제외)

In [66]:
# 한글/영어 문자가 아닌 숫자, 기호 제외
def apply_regular_expression(text, punc):
    # syonlp noun extractor 사용을 위해 문장사이 doublespace 삽입
    for ele in text:
      if ele in punc:
          text = text.replace(ele, "  ")
    pattern = re.compile('[^a-zA-Z| ㄱ-ㅣ 가-힣]')  # 추출 규칙: 영어 또는 띄어 쓰기(1 개)를 포함한 한글
    result = pattern.sub('', text)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    #result = result.translate(str.maketrans('', '', string.punctuation)) # punctuation marks 제외
    return result

In [67]:
punctuation = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

df1['Content'] = df1['Content'].apply(lambda x: apply_regular_expression(x,punctuation))
df2['Content'] = df2['Content'].apply(lambda x: apply_regular_expression(x,punctuation))
df3['Content'] = df3['Content'].apply(lambda x: apply_regular_expression(x,punctuation))

In [71]:
df1.loc[10,'Content']

'안녕하세요   수블린입니다   번째 용기내 챌린지는 배떡에서 로제떡볶이를 포장하기로 했어요  이전에 배떡에서 용기 가져가서 포장해온 경험이 있어요   갑자기 로제 떡볶이가 먹고 싶어서 집에서 제일 가까운 배떡에서 로제떡볶이 보통맛을 주문하기로 했어요  이미 배떡에서 용기를 가져가서 포장해 본 적이 있어서 가게로 전화하지 않고 요청사항에 개인 용기에 담아 간다고 작성을 해두었습니다   그리고 배떡 같은 경우에는 메뉴 선택하면서 단무지 필요 여부를 체크할 수 있어서 좋았어요   단무지 필요 없다고 해도 바쁘시면 단무지를 챙겨주시는 경우가 있더라고요  그래서 용기를 가지고 가기 때문에 후다닥 배떡 매장으로 갔어요  스타벅스 장바구니에 스테인리스 용기 개와 텀블러를 가지고 나갔어요   다른 것도 포장을 하기 위해 챙겨가는 센스 배떡에서 로제떡볶이 보통맛으로 주문해서 용기에 받아왔어요   제가 방문했을 때 딱 조리가 거의 끝난 상황이라서 아주 잠깐 기다렸다가 용기에 떡볶이를 받아서 나왔어요 배떡 로제떡볶이와 다른 곳에서 고기 군만두를 포장해서 같이 맛있게 잘 먹었어요  한 번에 다 먹기에는 양이 많아서 그릇에 먹을 만큼 덜어서 먹었어요   오랜만에 로제 떡볶이를 먹어서 너무너무 맛있게  잘 먹었어요 ㅠㅠ   제로웨이스트   제로웨이스트도전   용기내   용기내챌린지   다회용포장   다회용용기사용하기   떡볶이   떡복이용기내   떡볶이용기내챌린지   배떡'

#### PyKoSpacing

적용했음. 

전희원님이 개발한 PyKoSpacing은 한국어 띄어쓰기 패키지로 띄어쓰기가 되어있지 않은 문장을 띄어쓰기를 한 문장으로 변환해주는 패키지입니다. PyKoSpacing은 대용량 코퍼스를 학습하여 만들어진 띄어쓰기 딥 러닝 모델로 준수한 성능을 가지고 있습니다.

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-dyzpbxr8
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-dyzpbxr8
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit 7bd81fe9c55a54e1b8e6f6bd5931b87bec98784f
     |████████████████████████████████| 454.3 MB 20 kB/s 
     |████████████████████████████████| 4.0 MB 58.0 MB/s 
     |████████████████████████████████| 462 kB 88.3 MB/s 
     |████████████████████████████████| 1.2 MB 68.1 MB/s 
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2255830 sha256=9240eb709f2b79a04d3c9d0a38f41329e0de73a430cb923ec64a1feb1061cf50
  Stored in directory: /tmp/pip-ephem-wheel-cache-pijs5roh/wheels/9b/93/81/a2a7dc8c66ede5bf30634d20635f32b95eac7ca2ea8844058b
Successfully built pykospacing
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.39.0
    Uninstalling grpcio-1.39.0:
      Successfully uninstalled grpcio-1.3

In [ ]:
from pykospacing import Spacing

def fix_spacing(text):
  spacing = Spacing()
  return spacing(text) 

In [ ]:
dfcopy['Content'] = dfcopy['Content'].apply(lambda x: fix_spacing(x)) #헐... 35분 걸림. 다른방법을 찾아봐야겟다...

#### SOYNLP의 Normalizer활용

대화 데이터, 댓글 데이터에 등장하는 반복되는 이모티콘의 정리 및 한글, 혹은 텍스트만 남기기 위한 함수를 제공합니다.

예시)

emoticon_normalize('ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ쿠ㅜㅜㅜㅜㅜㅜ', num_repeats=3)

결과:  'ㅋㅋㅋㅜㅜㅜ'

In [73]:
!pip install soynlp

     |████████████████████████████████| 416 kB 7.9 MB/s 


In [74]:
from soynlp.normalizer import *

In [75]:
def normalize_all(df):
  for idx in df.index:
    df.loc[idx,'Content'] = emoticon_normalize(df.loc[idx,'Content'], num_repeats=3) 
    df.loc[idx,'Content'] = repeat_normalize(df.loc[idx,'Content'], num_repeats=2)
  return df

In [76]:
df_blog = normalize_all(df_blog)
df_insta = normalize_all(df_insta)
df_cafe = normalize_all(df_cafe)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

### 토큰화 - SOYNLP의 Noun Extractor(v.2) 활용

soynlp는 품사 태깅, 단어 토큰화들을 제공함.
비지도 학습으로 단어 토큰화 진행 - 데이터에서 자주 등장하는 단어들 (신조어 포함) 단어로 분석함.

#### Noun Extractor ver 2

soynlp=0.0.46+ 에서는 명사 추출기 version 2 를 제공합니다. 이전 버전의 명사 추출의 정확성과 합성명사 인식 능력, 출력되는 정보의 오류를 수정한 버전입니다. 

In [78]:
from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2

In [79]:
postings_blog ="".join( df_blog['Content'].tolist())
postings_insta ="".join( df_insta['Content'].tolist())
postings_cafe ="".join( df_cafe['Content'].tolist())

In [80]:
postings = [postings_blog, postings_insta, postings_cafe]

In [82]:
len(postings) # postings = [블로그 말뭉치, 인스타 말뭉치, 카페 말뭉치]

3

In [83]:
# min_noun_frequency default
noun_extractor = LRNounExtractor_v2(verbose=True, extract_compound=True)
noun_extractor.train(postings)
nouns = noun_extractor.extract()

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 367257 from 3 sents. mem=0.639 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=2105757, mem=1.394 Gb
[Noun Extractor] batch prediction was completed for 102235 words
[Noun Extractor] checked compounds. discovered 78476 compounds
[Noun Extractor] postprocessing detaching_features : 99673 -> 76133
[Noun Extractor] postprocessing ignore_features : 76133 -> 75826
[Noun Extractor] postprocessing ignore_NJ : 75826 -> 74814
[Noun Extractor] 74814 nouns (78476 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.615 Gb                    
[Noun Extractor] 69.50 % eojeols are covered


In [84]:
# min_noun_frequency=10 지정
nouns_minfreq = noun_extractor.train_extract(
    postings,
    min_noun_frequency=10
    )

[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 367257 from 3 sents. mem=1.721 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=2105757, mem=2.085 Gb
[Noun Extractor] batch prediction was completed for 102235 words
[Noun Extractor] checked compounds. discovered 78476 compounds
[Noun Extractor] postprocessing detaching_features : 11694 -> 11450
[Noun Extractor] postprocessing ignore_features : 11450 -> 11209
[Noun Extractor] postprocessing ignore_NJ : 11209 -> 11012
[Noun Extractor] 11012 nouns (78476 compounds) with min frequency=10
[Noun Extractor] flushing was done. mem=2.085 Gb                    
[Noun Extractor] 66.23 % eojeols are covered


In [86]:
nouns['용기내챌린지']

NounScore(frequency=170, score=1.0)

In [87]:
nouns_minfreq['용기내챌린지']

NounScore(frequency=170, score=1.0)

In [88]:
top100 = sorted(nouns_minfreq.items(), 
    key=lambda x:-x[1].frequency)[:100]

for i, (word, score) in enumerate(top100):
    if i % 5 == 0:
        print()
    print('%6s (%.2f)' % (word, score.score), end='')


     수 (0.90)    사용 (1.00)     것 (1.00)제로웨이스트 (0.96)  플라스틱 (1.00)
     잘 (0.40)     원 (0.97)    환경 (1.00)     위 (1.00)    너무 (0.93)
   com (0.85)     저 (0.81)     개 (0.94)     안 (0.80)     때 (0.88)
   친환경 (0.97)     등 (0.99)    함께 (0.91)    생각 (0.99)    실천 (1.00)
     대 (0.97)    참여 (1.00)    제품 (1.00)     월 (0.98)    가능 (1.00)
   그리고 (1.00)     중 (0.99)    제로 (0.96)    후기 (0.80)    정말 (0.95)
     명 (0.99)     통 (1.00)     년 (0.99)    지구 (1.00)   있어요 (0.86)
    우리 (0.97)     거 (0.47)    시간 (1.00)products (1.00)  웨이스트 (0.98)
     제 (0.99)   이렇게 (0.78)   수세미 (0.97)    비누 (0.93)    위한 (0.60)
     또 (0.60)    구매 (0.99)     물 (0.82)    내가 (0.60)   쓰레기 (0.67)
    다양 (1.00)    요즘 (1.00)   재활용 (0.98)    조금 (1.00)     집 (0.87)
    필요 (1.00)     곳 (1.00)     못 (0.93)    사람 (0.99)    오늘 (0.99)
    시작 (1.00)    바로 (1.00)     맛 (0.96)     분 (0.99)    다시 (0.81)
     책 (0.99)     후 (0.70)   그래서 (1.00)    방법 (1.00)    사진 (1.00)
     번 (0.81)    카페 (0.92)    모두 (0.72)   때문에 (0.60)     말 (0.55)
     시 

### stopwords 처리

In [89]:
stopwords=[]
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords[:10]

[['휴'],
 ['아이구'],
 ['아이쿠'],
 ['아이고'],
 ['어'],
 ['나'],
 ['우리'],
 ['저희'],
 ['따라'],
 ['의해']]

TODO: stopwords에서 '우리', '저희' 는 빼야하는것같다. 제로웨이스트의 공동체 성격을 유지하기위해서.

In [90]:
type(stopwords)

list

In [91]:
stopwords.remove(['우리'])

In [92]:
stopwords.remove(['저희'])

In [93]:
stopwords.remove(['함께'])

In [94]:
len(stopwords)

671

In [95]:
zerowaste_stopwords = ['제로','웨','이스트','이스터','제로 웨이스트','제로웨이스트',\
                       '제로 웨이스터','제로웨이스터','웨이스트','웨이스트 ', ' 웨이스트 ',' 웨이스트',\
                       '환경','친환경','너무', '그리고', '정말', '이렇게', '있어요', '때문', \
                       '정도', '조금', '분들', '진짜', '대한', '이번', '경우', '대신', '가지고',\
                       '그래서','엄청','아직']

In [96]:
for word in zerowaste_stopwords:
  stopwords.append(word)

In [97]:
include = ['낮', '땅', '밭', '꽃', '돌', '멋','맛', '폼', '물', '볕', '빛', '봄', '숲', '새', '산', '숨', '싹', '옷', '잎', '차', '흙', '힘']

In [99]:
# 한글자 키워드 제거 (include 리스트에 있는 한글자 단어는 포함시키도록 조정)
filtered_nouns1=[]

for extract in nouns_minfreq.items():
  x = extract[0]
  if (len(x) > 1 or  x in include): 
    filtered_nouns1.append((x, extract[1]))

In [100]:
# 불용어 제거
filtered_nouns=[]

for extract2 in filtered_nouns1:
  x2 = extract2[0]
  if x2 not in stopwords: 
    filtered_nouns.append((x2, extract2[1]))

### extract된 nouns 저장

In [101]:
import pickle

In [102]:
file_to_store = open("gdrive/My Drive/TeamProject2108/extracted_nouns_all.pkl", "wb")
pickle.dump(filtered_nouns,file_to_store)

file_to_store.close()

### 빈도수 높은 nouns 추출

In [103]:
top100 = sorted(filtered_nouns, 
    key=lambda x:-x[1][0])[:100]

for i, (word, score) in enumerate(top100):
    if i % 5 == 0:
        print()
    print('%6s (%.2f)' % (word, score.frequency), end='')


    사용 (12325.00)  플라스틱 (6842.00)   com (5069.00)    함께 (4237.00)    생각 (3948.00)
    실천 (3897.00)    참여 (3750.00)    제품 (3616.00)    가능 (3513.00)    후기 (3261.00)
    지구 (3117.00)    우리 (3004.00)    시간 (2969.00)products (2896.00)   수세미 (2733.00)
    비누 (2655.00)    위한 (2637.00)    구매 (2575.00)     물 (2563.00)    내가 (2539.00)
   쓰레기 (2523.00)    다양 (2514.00)    요즘 (2456.00)   재활용 (2377.00)    필요 (2335.00)
    사람 (2277.00)    오늘 (2245.00)    시작 (2236.00)    바로 (2210.00)     맛 (2199.00)
    다시 (2187.00)    방법 (2099.00)    사진 (2064.00)    카페 (2006.00)    모두 (1994.00)
   때문에 (1978.00)    지금 (1890.00)   텀블러 (1852.00)    칫솔 (1851.00)    가장 (1829.00)
    사실 (1785.00)   샴푸바 (1781.00)    진행 (1744.00)    비건 (1731.00)    포장 (1730.00)
    느낌 (1728.00)    자연 (1713.00)    관심 (1697.00)    직접 (1696.00)    빨대 (1684.00)
    비닐 (1644.00)    상품 (1586.00)   코로나 (1573.00)    물건 (1567.00)    것을 (1554.00)
    판매 (1551.00)    건강 (1550.00)    한번 (1548.00)    이상 (1523.00)   일회용 (1516.00)
    처음 (1481.00)    확인 (

In [104]:
top500 = sorted(filtered_nouns, 
    key=lambda x:-x[1][0])[:500]

for i, (word, score) in enumerate(top500):
    if i % 5 == 0:
        print()
    print('%6s (%.2f)' % (word, score.frequency), end='')


    사용 (12325.00)  플라스틱 (6842.00)   com (5069.00)    함께 (4237.00)    생각 (3948.00)
    실천 (3897.00)    참여 (3750.00)    제품 (3616.00)    가능 (3513.00)    후기 (3261.00)
    지구 (3117.00)    우리 (3004.00)    시간 (2969.00)products (2896.00)   수세미 (2733.00)
    비누 (2655.00)    위한 (2637.00)    구매 (2575.00)     물 (2563.00)    내가 (2539.00)
   쓰레기 (2523.00)    다양 (2514.00)    요즘 (2456.00)   재활용 (2377.00)    필요 (2335.00)
    사람 (2277.00)    오늘 (2245.00)    시작 (2236.00)    바로 (2210.00)     맛 (2199.00)
    다시 (2187.00)    방법 (2099.00)    사진 (2064.00)    카페 (2006.00)    모두 (1994.00)
   때문에 (1978.00)    지금 (1890.00)   텀블러 (1852.00)    칫솔 (1851.00)    가장 (1829.00)
    사실 (1785.00)   샴푸바 (1781.00)    진행 (1744.00)    비건 (1731.00)    포장 (1730.00)
    느낌 (1728.00)    자연 (1713.00)    관심 (1697.00)    직접 (1696.00)    빨대 (1684.00)
    비닐 (1644.00)    상품 (1586.00)   코로나 (1573.00)    물건 (1567.00)    것을 (1554.00)
    판매 (1551.00)    건강 (1550.00)    한번 (1548.00)    이상 (1523.00)   일회용 (1516.00)
    처음 (1481.00)    확인 (